<a href="https://colab.research.google.com/github/skasrap/NMA-DL/blob/main/NMA_DL_Project_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Setup and Imports


In [ ]:
#@title
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import scipy.io
from sklearn.metrics import average_precision_score, precision_recall_curve

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torchvision import transforms
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf #*
from tensorflow import keras #*
import time #*

print ('Import Completed')

def set_device():
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  if device == 'cuda':
    print('GPU yay!')
  elif device == 'cpu':
    print('CPU :|')
  return device
DEVICE = set_device()

Import Completed
GPU yay!


Load Images w/o resizing

In [ ]:
#@title
# image_dir = "/content/drive/MyDrive/Dataset/emotic/emodb_small/images"                      #Original
image_dir = "/content/drive/MyDrive/Neuromatch/Project Neuromatch/emotic/emodb_small/images" #Alternative
image_data = []
x = 0
print("Raw Image Shapes: ")
for filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, filename)
  img = Image.open(image_path)
  img_array = np.array(img)
  print(img_array.shape)
  # image_data.append(img_array.flatten()) # here we change the 3D numpy array to a 1D vector and then add it to the image_data list
  x += 1
  if x > 9: # First lets try to just get 10 images
    break


Raw Image Shapes: 


FileNotFoundError: ignored

Load Images and Resize Them

In [ ]:
#@title
# As seen, we got a small problem; the images don't have the same size. But we can resize them.
target_width = 224
target_height = 224
image_data = dict()

for filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, filename)
  img = Image.open(image_path)
  img_resized = np.array(img.convert('RGB').resize((target_width, target_height))) # we need to use convert() function to make sure even black and
  # white images are in RGB mode, to ensure coherent matrix dimensions!
  img_resized = img_resized.astype(np.float32) / 255.0 #Normalize the pixel values.
  if img_resized.ndim == 3:
    image_data[filename] = img_resized
  else:
    print(filename)
  # if len(image_data) == 100: # First lets try to just get 100 images
    # break

Get the Annotations & Labels

In [ ]:
#@title
# Now we must obtain the labels for each image
annotations_mat = "/content/drive/MyDrive/Dataset/Annotations/Annotations.mat"                       #Original
# annotations_mat = "/content/drive/MyDrive/Neuromatch/Project Neuromatch/Annotations/Annotations.mat"   #Alternative
annotations_py = scipy.io.loadmat(annotations_mat)

train_labels = annotations_py['train'][0, :]
# try dataloaders
Annotations = []
for x in train_labels:
  if x[1][0] == 'emodb_small/images':
    filename = x[0][0]
    i = x[4][0][0] # This is the MATLAB Nest corresponding to main data for each image.
    bbox = i[0][0].tolist()
    annotations_categories = []
    annotations_continuous = []
    for array in i[1][0][0][0][0]:
      annotations_categories.append(array[0])
    for array in i[2][0][0]:
      annotations_continuous.append(array[0][0])
    Annotations.append([filename, bbox, annotations_categories, annotations_continuous])
print(Annotations[0])
print(len(Annotations))

Emotion_Categories = ['Affection', 'Anger', 'Annoyance', 'Anticipation', 'Aversion', 'Confidence', 'Disapproval', 'Disconnection',
       'Disquietment', 'Doubt/Confusion', 'Embarrassment', 'Engagement', 'Esteem', 'Excitement', 'Fatigue', 'Fear',
       'Happiness', 'Pain', 'Peace', 'Pleasure', 'Sadness', 'Sensitivity', 'Suffering', 'Surprise', 'Sympathy', 'Yearning']
x = Annotations[0]
print(x)
Labels = []
for i in Annotations:
  y = [1. if elem in i[2] else 0. for elem in Emotion_Categories]
  Labels.append([i[0], y, i[3]])

  # Now we have to make the order of X (images) and Y (labels) same.
# image_data['ebpsbjqzfhd7efh00i.jpg'][0]
X = []
y_cat = []
y_cont = []
for instance in Labels:
  if instance[0] in image_data.keys():
    X.append(image_data[instance[0]])
    y_cat.append(instance[1])
    y_cont.append(instance[2])
# print(y_cat[0])
# print(y_cont[0])
# print(len(X))
# print(len(y_cat))
# print(len(y_cont))

Load the Train and Test Data

In [ ]:
X_tensor = torch.tensor(np.array(X))
y_cat_tensor = torch.tensor(y_cat)

In [ ]:
X_tensor = X_tensor.permute(0, 3, 1, 2).to(DEVICE)
y_cat_tensor = y_cat_tensor.to(DEVICE)
train_dataset = list(zip(X_tensor, y_cat_tensor))
train_loader = DataLoader(train_dataset, 10, shuffle = True)

NameError: ignored

Train a Basic Neural Network

In [ ]:
AlexNet = models.alexnet(pretrained = True)
num_classes = 26

for param in AlexNet.parameters():
  param.requires_grad = False

in_features = AlexNet.classifier[-1].in_features
AlexNet.classifier[-1] = nn.Linear(in_features, num_classes).to(DEVICE)
AlexNet.classifier.add_module('sigmoid', nn.Sigmoid())
criterion = nn.BCELoss()

optimizer = optim.Adam(AlexNet.parameters(), lr=0.001)

AlexNet.to(DEVICE)
# Step 6: Training Loop
num_epochs = 10000
losses = []
accuracies = []

for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        # Move data to the GPU if available
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = AlexNet(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)
        correct_predictions += (outputs > 0.5).eq(labels > 0.5).sum().item()

    # Calculate epoch-level loss and accuracy
    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")


#Plot Loss and Accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(accuracies)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.show()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10000] - Loss: 0.3625, Accuracy: 22.8362
Epoch [2/10000] - Loss: 0.2948, Accuracy: 23.2094
Epoch [3/10000] - Loss: 0.2692, Accuracy: 23.3424
Epoch [4/10000] - Loss: 0.2516, Accuracy: 23.4458
Epoch [5/10000] - Loss: 0.2341, Accuracy: 23.6305
Epoch [6/10000] - Loss: 0.2186, Accuracy: 23.7155
Epoch [7/10000] - Loss: 0.2067, Accuracy: 23.8978
Epoch [8/10000] - Loss: 0.2001, Accuracy: 23.9360
Epoch [9/10000] - Loss: 0.1920, Accuracy: 24.0012
Epoch [10/10000] - Loss: 0.1800, Accuracy: 24.1502
Epoch [11/10000] - Loss: 0.1751, Accuracy: 24.2007
Epoch [12/10000] - Loss: 0.1683, Accuracy: 24.2475
Epoch [13/10000] - Loss: 0.1628, Accuracy: 24.2857
Epoch [14/10000] - Loss: 0.1556, Accuracy: 24.4138
Epoch [15/10000] - Loss: 0.1559, Accuracy: 24.4163
Epoch [16/10000] - Loss: 0.1478, Accuracy: 24.4791
Epoch [17/10000] - Loss: 0.1440, Accuracy: 24.5603
Epoch [18/10000] - Loss: 0.1423, Accuracy: 24.5603
Epoch [19/10000] - Loss: 0.1403, Accuracy: 24.5899
Epoch [20/10000] - Loss: 0.1357, Accurac

KeyboardInterrupt: ignored